<a href="https://colab.research.google.com/github/eaadeniyi/acolite/blob/main/acolite_gee_atmCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Upload ACOLITE to Google Drive (One-time, 10 minutes)
*Clone directly to Drive (RECOMMENDED)*`

In [ ]:
# Run this in a Colab cell
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/acolite/acolite.git /content/drive/MyDrive/ACOLITE
print("✓ ACOLITE installed to Google Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path '/content/drive/MyDrive/ACOLITE' already exists and is not an empty directory.
✓ ACOLITE installed to Google Drive


Authenticate Google Earth Engine

In [ ]:
import ee

# This will open a browser window for authentication
ee.Authenticate()

# After authentication, initialize
ee.Initialize(project='eaadeniyi')
print("✓ Earth Engine ready")

✓ Earth Engine ready


In [ ]:
# Initialize with your project
ee.Initialize(project='eaadeniyi')
print("✓ Success!")

✓ Success!


In [ ]:
test_id = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').first().id().getInfo()
print("✓ EE connection ok; example asset:", test_id)


✓ EE connection ok; example asset: 20150627T102531_20160606T223605_T31RCL


`Install dependencies`

In [ ]:
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.1 MB/s eta 0:00:00


In [ ]:
!pip -q install --upgrade earthengine-api
!earthengine set_project eaadeniyi
import ee
ee.Authenticate()
ee.Initialize(project='eaadeniyi')


import sys, os
acolite_path = '/content/drive/MyDrive/ACOLITE'  # repo root that contains the folder "acolite/"
assert os.path.isdir(acolite_path), f"Not found: {acolite_path}"
assert os.path.isdir(os.path.join(acolite_path, 'acolite')), "Folder 'acolite/' missing inside the repo"

if acolite_path not in sys.path:
    sys.path.insert(0, acolite_path)   # add repo root so 'import acolite' works

import acolite
from acolite import gee as acolite_gee
print("✓ Imported ACOLITE from", acolite_path)



print("=== DIAGNOSTIC ===")
print(f"EE module location: {ee.__file__}")

try:
    ee.Initialize(project='eaadeniyi')
    print("✓ Direct initialization works")

    # Try a simple query
    img = ee.Image('COPERNICUS/S2_HARMONIZED/20150627T102531_20160606T223605_T31RCL')
    props = img.getInfo()
    print(f"✓ Can access images: {props['type']}")

    # Try finding scenes
    from acolite import gee as acolite_gee
    images, imColl = acolite_gee.find_scenes(
        '2024-06-15',
        isodate_end='2024-06-15',
        sensors=['S2A_MSI', 'S2B_MSI'],
        limit=[29.67461471404195, -90.17624766298297, 29.971278012572437, -89.78348643251422],
        surface_reflectance=False
    )
    print(f"✓ Can search scenes: found {len(images)} images")

    print("\n✓ ALL TESTS PASSED - main() should work now!")

except Exception as e:
    print(f"\n✗ Error at diagnostic: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
"""
PRODUCTION-READY ACOLITE-GEE PROCESSING SYSTEM
Mississippi River Turbidity Monitoring (2017-2025)
155 Target Dates - Optimized for Google Colab

Author: Custom Configuration for BelleChasse Region
Date: 2025
"""

import ee
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import time

!pip -q install --upgrade earthengine-api
!earthengine set_project eaadeniyi
#import ee
ee.Authenticate()
ee.Initialize(project='eaadeniyi')


acolite_path = '/content/drive/MyDrive/ACOLITE'  # repo root that contains the folder "acolite/"
assert os.path.isdir(acolite_path), f"Not found: {acolite_path}"
assert os.path.isdir(os.path.join(acolite_path, 'acolite')), "Folder 'acolite/' missing inside the repo"

if acolite_path not in sys.path:
    sys.path.insert(0, acolite_path)   # add repo root so 'import acolite' works

import acolite
from acolite import gee as acolite_gee
print("✓ Imported ACOLITE from", acolite_path)


# ============================================================================
# CONFIGURATION SECTION - CUSTOMIZE HERE
# ============================================================================

# Your complete list of 155 target dates
TARGET_DATES = [
    '2022-03-31'
  #'2018-05-08'


]

# Sort dates chronologically
TARGET_DATES = sorted(TARGET_DATES)

# Region configuration
REGION_CONFIG = {
    'limit': [29.67461471404195, -90.17624766298297,
              29.971278012572437, -89.78348643251422],
    'region_name': 'BelleChasse-Region',
    'center_lat': 29.822946,
    'center_lon': -89.979865,
}

# Processing configuration
PROCESSING_CONFIG = {
    'sensors': ['S2A_MSI', 'S2B_MSI'],
    'batch_size': 20,  # Process 20 scenes before checkpoint
}

# Output configuration
OUTPUT_CONFIG = {
    'base_dir': '/content/drive/MyDrive/ACOLITE_Output_Production',
    'organize_by_year': True,  # Create subdirectories by year
}

# ============================================================================
# GOOGLE COLAB SETUP
# ============================================================================

def setup_environment():
    """
    Complete environment setup for Google Colab
    """
    print("="*80)
    print("ACOLITE-GEE PRODUCTION SYSTEM - ENVIRONMENT SETUP")
    print("="*80)

    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✓ Google Drive mounted")
    except:
        print("⚠ Drive already mounted or not in Colab")

    # Setup ACOLITE
    acolite_path = '/content/drive/MyDrive/ACOLITE'

    if not os.path.exists(acolite_path):
        print("\n⚠ ACOLITE not found on Drive. Cloning...")
        os.system(f'git clone https://github.com/acolite/acolite.git {acolite_path}')
        print("✓ ACOLITE cloned to Drive")

    if acolite_path not in sys.path:
        sys.path.insert(0, acolite_path)

    # Import ACOLITE
    import acolite as ac
    from acolite import gee
    print(f"✓ ACOLITE loaded from: {acolite_path}")

    # Initialize GEE - FIXED VERSION
    print("\nInitializing Google Earth Engine...")

    # Just initialize directly - authentication should already be done
    ee.Initialize(project='eaadeniyi')
    print("✓ Earth Engine initialized with project 'eaadeniyi'")

    # Create output directories
    os.makedirs(OUTPUT_CONFIG['base_dir'], exist_ok=True)
    if OUTPUT_CONFIG['organize_by_year']:
        years = set([d[:4] for d in TARGET_DATES])
        for year in years:
            year_dir = f"{OUTPUT_CONFIG['base_dir']}/{year}"
            os.makedirs(year_dir, exist_ok=True)
        print(f"✓ Created output directories for years: {sorted(years)}")

    print("="*80)
    print("✓ ENVIRONMENT SETUP COMPLETE\n")

    return ac, gee

# ============================================================================
# DATE ANALYSIS AND VALIDATION
# ============================================================================

def analyze_target_dates(target_dates):
    """
    Analyze target dates and provide statistics
    """
    dates = pd.to_datetime(target_dates)

    print("="*80)
    print("TARGET DATES ANALYSIS")
    print("="*80)
    print(f"These are EXACT Sentinel-2 acquisition dates (no tolerance)")

    # Overall statistics
    print(f"\nTotal target dates: {len(dates)}")
    print(f"Date range: {dates.min().date()} to {dates.max().date()}")
    print(f"Time span: {(dates.max() - dates.min()).days} days ({(dates.max() - dates.min()).days/365.25:.1f} years)")

    # Check for future dates
    today = pd.Timestamp.now()
    future_dates = dates[dates > today]
    if len(future_dates) > 0:
        print(f"\n⚠ WARNING: {len(future_dates)} dates are in the future:")
        for fd in future_dates[:5]:
            print(f"  - {fd.date()}")
        if len(future_dates) > 5:
            print(f"  ... and {len(future_dates)-5} more")

    # Distribution by year
    print("\nDistribution by year:")
    year_counts = dates.year.value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"  {year}: {count} dates")

    # Distribution by month
    print("\nDistribution by month:")
    month_counts = dates.month.value_counts().sort_index()
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for month, count in month_counts.items():
        print(f"  {month_names[month-1]}: {count} dates")

    # Check Sentinel-2 availability
    s2a_start = pd.Timestamp('2017-03-28')
    pre_s2a = dates[dates < s2a_start]
    if len(pre_s2a) > 0:
        print(f"\n⚠ WARNING: {len(pre_s2a)} dates before Sentinel-2A:")
        for pd_date in pre_s2a:
            print(f"  - {pd_date.date()}")

    print("="*80 + "\n")

    return {
        'total': len(dates),
        'future': len(future_dates),
        'pre_s2a': len(pre_s2a),
        'processable': len(dates) - len(future_dates) - len(pre_s2a),
        'years': sorted(year_counts.index.tolist()),
        'year_counts': year_counts.to_dict()
    }

# ============================================================================
# OPTIMIZED SETTINGS
# ============================================================================

def get_processing_settings(output_year_dir=None):
    """
    Get optimized processing settings
    Balances speed and quality as requested
    """
    output_dir = output_year_dir if output_year_dir else OUTPUT_CONFIG['base_dir']

    settings = {
        # Geographic
        'limit': REGION_CONFIG['limit'],
        'region_name': REGION_CONFIG['region_name'],
        'st_lat': REGION_CONFIG['center_lat'],
        'st_lon': REGION_CONFIG['center_lon'],
        'polygon_limit': True,
        'st_crop': False,
        'st_box': 20,
        'subset_aot': False,
        'strict_subset': False,
        'region_name_add_box': False,
        'add_region_name_output': False,

        # Sensors and date
        'sensors': PROCESSING_CONFIG['sensors'],
        'surface_reflectance': False,
        'isodate_start': None,  # Will be set per scene
        'isodate_end': None,    # Will be set per scene
        'day_range': 1,
        'filter_tiles': None,

        # Resolution
        's2_target_res': 10,
        'output_scale': 10,

        # Atmospheric Correction - OPTIMIZED FOR SPEED + QUALITY
        'atmospheric_correction': True,
        'run_hybrid_dsf': True,  # Use DSF for quality

        # DSF optimization (balance speed and quality)
        'percentiles': [5, 25, 75],  # 3 percentiles (faster than 6)
        'pidx': 1,  # Use 25th percentile for turbid waters
        'nbands': 2,  # Use 2 bands (faster than 3)
        'sel_par': 'taua_cv',

        # Glint correction (critical for water)
        'glint_correction': True,
        'glint_min': 0.0,
        'glint_max': 0.05,
        'glint_wind': 2.0,

        # Ancillary data
        'ancillary_data': True,
        'uoz_default': 0.3,
        'uwv_default': 1.5,
        'pressure': 1013.25,
        'wind': None,

        # Atmospheric correction parameters
        'rhop_par': 'romix',

        # TACT parameters (required even if not using)
        'run_hybrid_tact': False,
        'reptran': 'reptran_fine',
        'source': 'era5',
        'emissivity': 'water',  # ← MISSING KEY - ADDED

        # Offline processing
        'run_offline_dsf': False,
        'run_offline_tact': False,

        # Output settings
        'output': output_dir,
        'convert_output': True,
        'store_output_locally': True,
        'store_output_google_drive': False,

        # What to store
        'store_rhot': True,
        'store_rhos': True,
        'store_geom': True,
        'store_sr': False,
        'store_st': False,
        'store_sp': False,

        # File management
        'l1r_delete_netcdf': False,
        'l2w_export_geotiff': False,
        'clear_output_zip_files': True,
        'override': False,
        'use_scene_name': False,

        # Processing optimization
        'tile_size': [512, 512],  # Balanced for speed and memory

        # Tasks
        'task_check_sleep': 10,
    }

    return settings

# ============================================================================
# SCENE FINDING WITH PROGRESS TRACKING
# ============================================================================

def find_all_scenes_for_dates(ac, gee, target_dates, show_progress=True):
    """
    Find scenes for EXACT target dates (no tolerance, no cloud filtering)
    These are known Sentinel-2 acquisition dates
    """
    print("="*80)
    print("SEARCHING FOR SENTINEL-2 SCENES")
    print("="*80)
    print(f"Target dates: {len(target_dates)}")
    print("Searching for EXACT dates (no tolerance, no cloud filtering)")
    print("="*80 + "\n")

    all_scenes = []
    dates_with_scenes = 0
    dates_without_scenes = 0

    for idx, target_date in enumerate(target_dates, 1):
        if show_progress:
            print(f"[{idx}/{len(target_dates)}] Searching: {target_date}...", end=' ')

        try:
            # Parse date
            dt = datetime.strptime(target_date, '%Y-%m-%d')

            # Check if date is in future
            if dt > datetime.now():
                print("⚠ Future date - skipping")
                continue

            # Check if before S2A
            if dt < datetime(2017, 3, 28):
                print("⚠ Before Sentinel-2A - skipping")
                continue

            # Search for EXACT date only (no tolerance)
            date_start = target_date
            date_end = (dt + timedelta(days=1)).strftime('%Y-%m-%d')

            # Search GEE
            images, imColl = gee.find_scenes(
                date_start,
                isodate_end=date_end,
                sensors=PROCESSING_CONFIG['sensors'],
                limit=REGION_CONFIG['limit'],
                surface_reflectance=False
            )

            if len(images) == 0:
                print("✗ No scenes found")
                dates_without_scenes += 1
                continue

            # Process each image found
            found_for_date = False
            for fkey, pid in images:
                try:
                    img = imColl.filter(ee.Filter.eq(fkey, pid)).first()
                    info = img.getInfo()
                    props = info['properties']

                    if 'PRODUCT_ID' not in props:
                        continue

                    # Extract date from PRODUCT_ID
                    product_id = props['PRODUCT_ID']

                    try:
                        date_str = product_id.split('_')[2][:8]
                        acq_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
                    except:
                        acq_date = target_date

                    # Get other metadata safely
                    cloud_cover = float(props.get('CLOUDY_PIXEL_PERCENTAGE', -999))

                    scene_info = {
                        'target_date': target_date,
                        'acquisition_date': acq_date,
                        'days_from_target': 0,  # ← FIXED: Added this field
                        'product_id': pid,
                        'satellite': pid[0:3],
                        'tile': props.get('MGRS_TILE', ''),
                        'cloud_cover': cloud_cover,
                        'solar_zenith': float(props.get('MEAN_SOLAR_ZENITH_ANGLE', -999)),
                        'solar_azimuth': float(props.get('MEAN_SOLAR_AZIMUTH_ANGLE', -999)),
                        'processing_baseline': str(props.get('PROCESSING_BASELINE', '')),
                        'fkey': fkey,
                    }

                    all_scenes.append(scene_info)
                    found_for_date = True
                    print(f"✓ {pid[:30]}... ({cloud_cover:.1f}% clouds)")

                except Exception as e:
                    print(f"⚠ Error processing image: {str(e)}")
                    continue

            if found_for_date:
                dates_with_scenes += 1
            else:
                dates_without_scenes += 1

        except Exception as e:
            print(f"✗ Error: {str(e)}")
            dates_without_scenes += 1

    # Create DataFrame
    df = pd.DataFrame(all_scenes)

    print("\n" + "="*80)
    print("SCENE SEARCH SUMMARY")
    print("="*80)
    print(f"Target dates searched: {len(target_dates)}")
    print(f"Dates with scenes: {dates_with_scenes}")
    print(f"Dates without scenes: {dates_without_scenes}")
    print(f"Total scenes found: {len(df)}")

    if len(df) > 0:
        print(f"\nCloud cover range: {df['cloud_cover'].min():.1f}% - {df['cloud_cover'].max():.1f}%")
        print(f"Mean cloud cover: {df['cloud_cover'].mean():.1f}%")

        # Check if multiple images per date
        dates_with_multiple = df.groupby('target_date').size()
        multi_dates = dates_with_multiple[dates_with_multiple > 1]
        if len(multi_dates) > 0:
            print(f"\n⚠ {len(multi_dates)} dates have multiple images (different tiles):")
            for date, count in multi_dates.items():
                tiles = df[df['target_date']==date]['tile'].unique()
                print(f"  {date}: {count} images from tiles {', '.join(tiles)}")

    print("="*80 + "\n")

    # Save scene inventory
    if len(df) > 0:
        inventory_path = f"{OUTPUT_CONFIG['base_dir']}/scene_inventory.csv"
        df.to_csv(inventory_path, index=False)
        print(f"✓ Scene inventory saved: {inventory_path}\n")

    return df

# ============================================================================
# BATCH PROCESSING WITH CHECKPOINTING
# ============================================================================

def process_scenes_in_batches(ac, gee, scenes_df, settings, batch_size=20, start_idx=0):
    """
    Process scenes in batches with automatic checkpointing
    """
    print("="*80)
    print("BATCH PROCESSING WITH TURBIDITY CALCULATION")
    print("="*80)
    print(f"Total scenes: {len(scenes_df)}")
    print(f"Batch size: {batch_size}")
    print(f"Starting from: {start_idx}")
    print(f"Estimated time: {len(scenes_df) * 90 / 3600:.1f} hours")
    print("="*80 + "\n")

    # Initialize storage
    rsrd, lutd, luti = {}, {}, {}
    results = []

    # Get ImageCollection once
    print("Initializing GEE ImageCollection...")
    _, imColl = gee.find_scenes(
        scenes_df['acquisition_date'].min(),
        isodate_end=scenes_df['acquisition_date'].max(),
        sensors=settings['sensors'],
        limit=settings['limit'],
        surface_reflectance=False
    )
    print("✓ ImageCollection ready\n")

    # Process each scene
    for idx in range(start_idx, len(scenes_df)):
        scene = scenes_df.iloc[idx]

        # Update output directory for year-based organization
        if OUTPUT_CONFIG['organize_by_year']:
            year = scene['acquisition_date'][:4]
            settings['output'] = f"{OUTPUT_CONFIG['base_dir']}/{year}"

        print(f"\n{'='*80}")
        print(f"[{idx+1}/{len(scenes_df)}] PROCESSING: {scene['product_id']}")
        print(f"{'='*80}")
        print(f"Target date:     {scene['target_date']}")
        print(f"Acquisition:     {scene['acquisition_date']} (exact date)")  # ← CHANGED
        print(f"Tile:            {scene['tile']}")
        print(f"Cloud cover:     {scene['cloud_cover']:.1f}%")
        print(f"Solar zenith:    {scene['solar_zenith']:.2f}°")

        try:
            # Process with ACOLITE
            image = (scene['fkey'], scene['product_id'])
            t0 = time.time()

            ret = gee.agh(image, imColl, rsrd=rsrd, lutd=lutd, luti=luti, settings=settings)

            t1 = time.time()
            processing_time = t1 - t0

            if not ret or len(ret) == 0:
                raise ValueError("No output generated")

            nc_file = list(ret.values())[0]
            print(f"\n✓ Processing completed in {processing_time:.1f}s")
            print(f"✓ Output: {os.path.basename(nc_file)}")

            # Load and calculate turbidity
            print(f"  Calculating Nechad 2016 turbidity...")

            # Open, read, and close immediately
            import xarray as xr
            ds = xr.open_dataset(nc_file)
            rhos_665 = ds['rhos_665'].values
            aot_550 = float(ds.attrs.get('ac_aot_550', -999))
            ac_model = str(ds.attrs.get('ac_model', 'N/A'))
            ds.close()  # CRITICAL: Close immediately after reading

            # Calculate turbidity
            A_t, C_t = 610.94, 0.2324
            turbidity = A_t * (rhos_665 / (1 - rhos_665 / C_t))
            turbidity = np.where((rhos_665 < 0.001) | (rhos_665 > 0.3), np.nan, turbidity)

            # Calculate statistics
            turb_stats = {
                'mean': float(np.nanmean(turbidity)),
                'std': float(np.nanstd(turbidity)),
                'median': float(np.nanmedian(turbidity)),
                'min': float(np.nanmin(turbidity)),
                'max': float(np.nanmax(turbidity)),
                'valid_pct': float((~np.isnan(turbidity)).sum() / turbidity.size * 100)
            }

            # Write turbidity back to NetCDF using ACOLITE
            turb_attrs = {
                'algorithm': 'Nechad et al. 2016',
                'band': '665nm',
                'units': 'FNU',
                'A_t': A_t,
                'C_t': C_t
            }
            ac.output.nc_write(nc_file, 'tur_nechad2016', turbidity,
                              dataset_attributes=turb_attrs)

            print(f"  ✓ Turbidity: {turb_stats['mean']:.2f} FNU (mean)")
            print(f"    Range: {turb_stats['min']:.2f} - {turb_stats['max']:.2f} FNU")
            print(f"    Valid: {turb_stats['valid_pct']:.1f}%")

            # Store results
            result = scene.to_dict()
            result.update({
                'processing_time_s': processing_time,
                'output_file': nc_file,
                'turbidity_mean': turb_stats['mean'],
                'turbidity_std': turb_stats['std'],
                'turbidity_median': turb_stats['median'],
                'turbidity_min': turb_stats['min'],
                'turbidity_max': turb_stats['max'],
                'valid_pixels_pct': turb_stats['valid_pct'],
                'aot_550': aot_550,
                'ac_model': ac_model,
                'status': 'success',
                'error': ''
            })
            results.append(result)

        except Exception as e:
            print(f"\n✗ ERROR: {str(e)}")
            # ... rest of error handling ...
            print(f"\n✗ ERROR: {str(e)}")
            result = scene.to_dict()
            result.update({
                'status': 'failed',
                'error': str(e),
                'processing_time_s': 0,
                'output_file': '',
                'turbidity_mean': np.nan
            })
            results.append(result)

            import traceback
            traceback.print_exc()

        # Checkpoint: Save every batch_size scenes
        if (idx + 1) % batch_size == 0 or (idx + 1) == len(scenes_df):
            checkpoint_df = pd.DataFrame(results)
            checkpoint_path = f"{OUTPUT_CONFIG['base_dir']}/processing_checkpoint_{idx+1}.csv"
            checkpoint_df.to_csv(checkpoint_path, index=False)

            success_count = (checkpoint_df['status'] == 'success').sum()
            print(f"\n{'='*80}")
            print(f"💾 CHECKPOINT SAVED")
            print(f"{'='*80}")
            print(f"Processed: {idx+1}/{len(scenes_df)} scenes")
            print(f"Success: {success_count} | Failed: {len(checkpoint_df) - success_count}")
            print(f"Checkpoint: {checkpoint_path}")
            print(f"{'='*80}\n")

            # Memory cleanup
            import gc
            gc.collect()

    # Final save
    results_df = pd.DataFrame(results)
    final_path = f"{OUTPUT_CONFIG['base_dir']}/processing_results_final.csv"
    results_df.to_csv(final_path, index=False)

    print(f"\n{'='*80}")
    print("BATCH PROCESSING COMPLETE")
    print(f"{'='*80}")
    print(f"Total processed: {len(results_df)}")
    print(f"Successful: {(results_df['status']=='success').sum()}")
    print(f"Failed: {(results_df['status']=='failed').sum()}")
    print(f"Final results: {final_path}")
    print(f"{'='*80}\n")

    return results_df

# ============================================================================
# MAIN EXECUTION FUNCTION
# ============================================================================

def main():
    """
    Main execution function - Run everything
    """
    print("\n" + "="*80)
    print("ACOLITE-GEE PRODUCTION PROCESSING SYSTEM")
    print("Mississippi River Turbidity Monitoring")
    print("BelleChasse Region (2017-2025)")
    print("="*80 + "\n")

    # Step 1: Setup
    ac, gee = setup_environment()

    # Step 2: Analyze dates
    date_stats = analyze_target_dates(TARGET_DATES)

    # Step 3: Find scenes
    scenes_df = find_all_scenes_for_dates(ac, gee, TARGET_DATES)

    # Restrict/Define the tile for the image processing all tiles increases processing time
    # keep only one tile globally (e.g., 15RYP)
    preferred_tile = PROCESSING_CONFIG.get("preferred_tile", "15RYP")
    keep = scenes_df['tile'] == preferred_tile
    if keep.any():
        dropped = len(scenes_df) - keep.sum()
        scenes_df = scenes_df[keep].copy()
        print(f"Filtered to tile {preferred_tile}. Dropped {dropped} other-tile scene(s).")
    else:
        print(f"No scenes in tile {preferred_tile}; keeping all tiles for now.")

    # Optionally: still ensure one scene per date (if needed)
    scenes_df = (scenes_df
        .sort_values(['target_date','days_from_target','cloud_cover'])
        .groupby('target_date', as_index=False)
        .first()
    )

    if len(scenes_df) == 0:
        print("⚠ No scenes found. Exiting.")
        return None

    # Step 4: Get processing settings
    settings = get_processing_settings()

    # Step 5: Process scenes
    results_df = process_scenes_in_batches(
        ac, gee, scenes_df, settings,
        batch_size=PROCESSING_CONFIG['batch_size'],
        start_idx=0
    )

    # Step 6: Summary
    print("\n" + "="*80)
    print("FINAL SUMMARY")
    print("="*80)
    print(f"Target dates: {len(TARGET_DATES)}")
    print(f"Scenes found: {len(scenes_df)}")
    print(f"Successfully processed: {(results_df['status']=='success').sum()}")
    print(f"Failed: {(results_df['status']=='failed').sum()}")

    if (results_df['status']=='success').sum() > 0:
        success_df = results_df[results_df['status']=='success']
        print(f"\nTurbidity statistics:")
        print(f"  Mean: {success_df['turbidity_mean'].mean():.2f} ± {success_df['turbidity_mean'].std():.2f} FNU")
        print(f"  Range: {success_df['turbidity_min'].min():.2f} - {success_df['turbidity_max'].max():.2f} FNU")
        print(f"\nProcessing time:")
        print(f"  Total: {success_df['processing_time_s'].sum()/3600:.1f} hours")
        print(f"  Average: {success_df['processing_time_s'].mean():.1f}s per scene")

    print(f"\nOutput directory: {OUTPUT_CONFIG['base_dir']}")
    print("="*80 + "\n")

    return results_df

# ============================================================================
# RUN THE SYSTEM
# ============================================================================

if __name__ == "__main__":
    results = main()

E0000 00:00:1761021537.853181   12881 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761021537.873169   12881 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761021537.938698   12881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761021537.938793   12881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761021537.938804   12881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761021537.938812   12881 computation_placer.cc:177] computation placer already registered. Please check linka